In [1]:
import ta
import pandas as pd
import sqlalchemy as sql
import yfinance as yf
import datetime as dt

In [2]:
engine = sql.create_engine('sqlite:///India')

In [3]:
names = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',engine)

In [4]:
names

,name
0,ASIANPAINT.BO
1,AXISBANK.BO
2,BAJAJ-AUTO.BO
3,BAJFINANCE.BO
4,BAJAJFINSV.BO
5,BHARTIARTL.BO
6,DRREDDY.BO
7,HCLTECH.BO
8,HDFC.BO
9,HDFCBANK.BO


In [5]:
names = names.name.to_list()
names

['ASIANPAINT.BO',
 'AXISBANK.BO',
 'BAJAJ-AUTO.BO',
 'BAJFINANCE.BO',
 'BAJAJFINSV.BO',
 'BHARTIARTL.BO',
 'DRREDDY.BO',
 'HCLTECH.BO',
 'HDFC.BO',
 'HDFCBANK.BO',
 'HINDUNILVR.BO',
 'ICICIBANK.BO',
 'INDUSINDBK.BO',
 'INFY.BO',
 'ITC.BO',
 'KOTAKBANK.BO',
 'LT.BO',
 'M&M.BO',
 'MARUTI.BO',
 'NESTLEIND.BO',
 'NTPC.BO',
 'ONGC.BO',
 'POWERGRID.BO',
 'RELIANCE.BO',
 'SBIN.BO',
 'SUNPHARMA.BO',
 'TCS.BO',
 'TECHM.BO',
 'TITAN.BO',
 'ULTRACEMCO.BO']

In [6]:
def Update_database(names):
    for name in names:
        df1 = yf.download(name,start=dt.datetime.now()).reset_index()
        df1.to_sql(name , engine , if_exists='append',index=False)
        print("Update successful..")

In [7]:
framelist = []

for name in names:
    framelist.append(pd.read_sql(f'SELECT Date, Close FROM "{name}"',engine))

In [8]:
framelist[0]

,Date,Close
0,2000-01-03 00:00:00.000000,24.371876
1,2000-01-04 00:00:00.000000,24.375000
2,2000-01-05 00:00:00.000000,24.125000
3,2000-01-06 00:00:00.000000,23.750000
4,2000-01-07 00:00:00.000000,23.937500
...,...,...
5675,2022-07-27 00:00:00.000000,3185.449951
5676,2022-07-28 00:00:00.000000,3263.199951
5677,2022-07-28 00:00:00.000000,3263.199951
5678,2022-07-28 00:00:00.000000,3263.199951


In [9]:
 def MACDdecision(df):
        df['MACD_diff']=ta.trend.macd_diff(df.Close)
        df.loc[(df.MACD_diff > 0)& (df.MACD_diff.shift(1) < 0),'Decision MACD'] = 'Buy'

In [10]:
def RSI_SMAdecision(df):
    df['RSI'] = ta.momentum.rsi(df.Close,window=10)
    df['SMA200'] = ta.trend.sma_indicator(df.Close,window=200)
    df.loc[(df.Close > df.SMA200) & (df.RSI < 30),'Decision RSI/SMA'] = 'Buy'

In [11]:
def MAC_RSI(framelist):
    for frame in framelist:
        MACDdecision(frame)
        RSI_SMAdecision(frame)
MAC_RSI(framelist)

In [12]:
framelist[0]

,Date,Close,MACD_diff,Decision MACD,RSI,SMA200,Decision RSI/SMA
0,2000-01-03 00:00:00.000000,24.371876,NaN,NaN,NaN,NaN,NaN
1,2000-01-04 00:00:00.000000,24.375000,NaN,NaN,NaN,NaN,NaN
2,2000-01-05 00:00:00.000000,24.125000,NaN,NaN,NaN,NaN,NaN
3,2000-01-06 00:00:00.000000,23.750000,NaN,NaN,NaN,NaN,NaN
4,2000-01-07 00:00:00.000000,23.937500,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5675,2022-07-27 00:00:00.000000,3185.449951,24.438729,NaN,83.025961,3089.917742,NaN
5676,2022-07-28 00:00:00.000000,3263.199951,28.953763,NaN,86.468597,3089.700741,NaN
5677,2022-07-28 00:00:00.000000,3263.199951,29.644799,NaN,86.468597,3089.495240,NaN
5678,2022-07-28 00:00:00.000000,3263.199951,27.835237,NaN,86.468597,3089.197490,NaN


In [13]:
def Buy(name,framelist):
    for name,frame in zip(names,framelist):
        if frame['Decision MACD'].iloc[-1] == 'Buy':
            print("Buying Signal MACD for "+name)
        if frame['Decision RSI/SMA'].iloc[-1] == 'Buy':
            print("Buying Signal RSI/SMA for "+name)
Buy(name,framelist)

In [14]:
Buy(name,framelist)

In [15]:
Update_database(names)
MAC_RSI(framelist)
Buy(name,framelist)

[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[*********************100%***********************]  1 of 1 completed
Update successful..
[********************